In [108]:
import requests
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split

In [66]:
def get_request_query(url, operation, params, serviceKey):
    import urllib.parse as urlparse
    params = urlparse.urlencode(params)
    request_query = url + '/' + operation + '?' + params + '&' + 'serviceKey' + '=' + serviceKey
    return request_query

In [141]:
# 요청 url과 오퍼레이션
URL = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService'
OPERATION = 'getHoliDeInfo' # 국경일 + 공휴일 정보 조회 오퍼레이션

# 파라미터
SERVICEKEY = 'Z5Sklpift0%2F0VXX874w7z3HJEF2lFrjyIcejRehRT3t3ajl31qVRnn2qUTxa%2BelVcj5ugQEkc8uM5g3mEGnmcw%3D%3D'
solYear = '2019'
solMonth = '02'

PARAMS = {'solYear' : solYear, 'solMonth' : solMonth}

In [142]:
request_query = get_request_query(URL, OPERATION, PARAMS, SERVICEKEY)
print('request_query: ', request_query)

response = requests.get(url=request_query)
print('status_code : ' + str(response.status_code))

request_query:  http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService/getHoliDeInfo?solYear=2019&solMonth=02&serviceKey=Z5Sklpift0%2F0VXX874w7z3HJEF2lFrjyIcejRehRT3t3ajl31qVRnn2qUTxa%2BelVcj5ugQEkc8uM5g3mEGnmcw%3D%3D
status_code : 200


In [143]:
soup = BeautifulSoup(response.content, 'lxml')

In [144]:
response

<Response [200]>

In [145]:
hol_list = soup.find_all('item')

In [146]:
for each_hol in hol_list:
    print(each_hol.find('datename').text, each_hol.find('locdate').text)

설날 20190204
설날 20190205
설날 20190206


In [150]:
# 연도별, 각 달별 공휴일 뽑아내기 

solYear_list = ['2019', '2018', '2017', '2016', '2015']
solMonth_list = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
URL = 'http://apis.data.go.kr/B090041/openapi/service/SpcdeInfoService'
OPERATION = 'getHoliDeInfo' # 국경일 + 공휴일 정보 조회 오퍼레이션
SERVICEKEY = 'Z5Sklpift0%2F0VXX874w7z3HJEF2lFrjyIcejRehRT3t3ajl31qVRnn2qUTxa%2BelVcj5ugQEkc8uM5g3mEGnmcw%3D%3D'

hol_dic = {}

for i, solYear in enumerate(solYear_list):
    for i, solMonth in enumerate(solMonth_list):
        
        PARAMS = {'solYear' : solYear, 'solMonth' : solMonth}
        request_query = get_request_query(URL, OPERATION, PARAMS, SERVICEKEY)
        response = requests.get(url=request_query)
        soup = BeautifulSoup(response.content, 'lxml')
        hol_list = soup.find_all('item')
        
        for i, each_hol in enumerate(hol_list):
            hol_dic[str(solYear + ' ' + each_hol.find('datename').text) + str(i)] = each_hol.find('locdate').text
        

In [177]:
len(hol_dic)

91

In [158]:
final_df = pd.read_csv('../../final_df_12/final_df_12.csv').drop(columns='Unnamed: 0')

In [159]:
final_df['release_date']

0       4/23/2015
1      12/17/2014
2       2/11/2015
3       6/11/2015
4       2/11/2015
          ...    
920    11/21/2019
921    11/20/2019
922    10/30/2019
923    10/16/2019
924     9/12/2019
Name: release_date, Length: 925, dtype: object

In [160]:
final_df['release_date'] = pd.to_datetime(final_df['release_date'])

In [161]:
final_df['release_date'] = final_df['release_date'].dt.strftime('%Y-%m-%d')

In [162]:
final_df['release_date']

0      20150423
1      20141217
2      20150211
3      20150611
4      20150211
         ...   
920    20191121
921    20191120
922    20191030
923    20191016
924    20190912
Name: release_date, Length: 925, dtype: object

In [175]:
final_df.head(1)

,title,director,distributor,distributor_group,release_date,release_season,release,form,nation,screen,...,director_power,sequel,original,actor_power,director_count,director_movie_count,actor_movie_count,log_aud,before_news,after_news
0,어벤져스: 에이지 오브 울트론,조스 웨던,월트디즈니컴퍼니코리아(주),B,20150423,spring,개봉영화,장편,미국,1843,...,7073037,1,0,33074337,1,1,11,16.166362,4120.0,1940.0


In [164]:
hol_arr = hol_dic.values()

In [165]:
holiday_movie_index = []

for holiday in hol_arr:
    for i, row in final_df.iterrows():
        if row['release_date'] == holiday:
            holiday_movie_index.append(i)

In [176]:
# list(holiday_movie_index)

In [169]:
df = final_df.iloc[holiday_movie_index]

In [217]:
len(df)

63

In [215]:
# list(df['title'])

# 공휴일에 개봉한 영화 구분 컬럼 추가

In [232]:
final_df['holiday'] = final_df['title'].apply(lambda title: 1 if title in list(df['title']) else 0)

In [233]:
# for i, row in df.iterrows():
#     final_df['holiday'][i] = 1
    

In [234]:
len(final_df[final_df['holiday'] == 1])

59

In [222]:
holiday_movie_index.sort()

In [229]:
# final_df.iloc[511]

In [228]:
# hol_dic

In [230]:
# np.array(holiday_movie_index)

In [203]:
# final_df[final_df['holiday'] == 1].index

Int64Index([ 15,  42,  85, 212, 226, 230, 281, 306, 314, 333, 345, 365, 381,
            385, 386, 388, 394, 400, 401, 426, 429, 441, 452, 473, 480, 485,
            511, 525, 547, 564, 566, 571, 582, 601, 605, 614, 651, 661, 662,
            668, 679, 691, 693, 695, 698, 700, 711, 723, 737, 738, 756, 792,
            813, 822, 839, 874, 900, 908, 924],
           dtype='int64')

In [231]:
# for index in list(holiday_movie_index):
#     if index not in final_df[final_df['holiday'] == 1].index:
#         print(index)

# 배급사 세개의 그룹으로 다시 그룹화

In [235]:
group_a = ['씨제이이앤엠(주)', '메가박스중앙(주)플러스엠', 'CGV아트하우스', '롯데쇼핑㈜롯데엔터테인먼트',
           'CGV아트하우스,(주)대명문화공장', '롯데쇼핑㈜롯데엔터테인먼트,(주)마인스 엔터테인먼트',
           'CGV아트하우스,(주)리틀빅픽쳐스', 'CGV아트하우스,영화사 풀', '메가박스중앙(주)플러스엠,(주)키위미디어그룹',
           '롯데쇼핑㈜롯데엔터테인먼트,(주)리틀빅픽쳐스', '메가박스중앙(주)플러스엠,주식회사 키위컴퍼니',
           '씨제이 씨지브이(CJ CGV)(주)', '롯데컬처웍스(주)롯데엔터테인먼트',
           '롯데컬처웍스(주)롯데엔터테인먼트,씨네그루(주)키다리이엔티', '메가박스중앙(주)플러스엠,(주)콘텐츠 난다긴다',
           '씨제이이앤엠(주),(주)덱스터스튜디오', '(주)쇼박스', '(주)넥스트엔터테인먼트월드(NEW)', '(주)쇼박스,(주)와우픽쳐스',
           '(주)넥스트엔터테인먼트월드(NEW),(주)팬엔터테인먼트']
group_b = ['월트디즈니컴퍼니코리아(주)', '이십세기폭스코리아(주)', '유니버설픽쳐스인터내셔널 코리아(유)',
           '워너브러더스 코리아(주)', '씨네그루(주)다우기술', '(주)이수C&E', '(주)티캐스트',
           '씨네그루(주)키다리이엔티', '월트디즈니컴퍼니코리아 유한책임회사', '소니픽쳐스엔터테인먼트코리아주식회사극장배급지점',
           '(주)이수C&E,(주)스톰픽쳐스코리아', '씨네그루(주)키다리이엔티,주식회사 키위컴퍼니',
           '(주)이수C&E,(주)에이원엔터테인먼트', '씨네그루(주)키다리이엔티,(주)삼백상회',
           '(주)이수C&E,(주)시너지미디어', '(주)셀트리온엔터테인먼트', '케이티하이텔(주)']

In [236]:
final_df['dist_cat'] = final_df['distributor']

for a in group_a:
    final_df['dist_cat'] = final_df['dist_cat'].replace(a, 'A')

for b in group_b:
    final_df['dist_cat'] = final_df['dist_cat'].replace(b, 'B')

final_df['dist_cat'] = final_df['dist_cat'].apply(
    lambda data: "C" if data != 'A' and data != 'B' else data)

In [245]:
len(final_df.columns)

35

In [259]:
final_df['sequel'][37] = 0
final_df['original'][37] = 1 

/Users/song-yeejun/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/song-yeejun/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
sequel                                     1
original 

In [260]:
final_df.iloc[37]

title                          그레이의 50가지 그림자
director                            샘 테일러-존슨
distributor              유니버설픽쳐스인터내셔널 코리아(유)
distributor_group                          B
release_date                        20150226
release_season                        winter
release                                 개봉영화
form                                      장편
nation                                    미국
screen                                   512
rev_nationwide                    2.9186e+09
aud_nationwide                        364415
rev_seoul                         1201564996
rev_seoul.1                           145292
genre                                    드라마
rate                                 청소년관람불가
type                                    일반영화
expert_rating                           3.07
starring_actors         제이미 도넌,다코타 존슨,루크 그림즈
running_times                            125
audience_rating                      5.57519
rating_audiences                         266
realease_y

In [261]:
final_df.columns

Index(['title', 'director', 'distributor', 'distributor_group', 'release_date',
       'release_season', 'release', 'form', 'nation', 'screen',
       'rev_nationwide', 'aud_nationwide', 'rev_seoul', 'rev_seoul.1', 'genre',
       'rate', 'type', 'expert_rating', 'starring_actors', 'running_times',
       'audience_rating', 'rating_audiences', 'realease_year',
       'director_power', 'sequel', 'original', 'actor_power', 'director_count',
       'director_movie_count', 'actor_movie_count', 'log_aud', 'before_news',
       'after_news', 'holiday', 'dist_cat'],
      dtype='object')

In [281]:
X = final_df.drop(columns=['aud_nationwide', 'log_aud'])
y = final_df[['aud_nationwide', 'log_aud']]

In [282]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [283]:
len(X_train), len(X_test), len(y_train), len(y_test)

(740, 185, 740, 185)

In [284]:
X_train.to_csv('X_train_13.csv')
X_test.to_csv('X_test_13.csv')
y_train.to_csv('y_train_13.csv')
y_test.to_csv('y_test_13.csv')

In [285]:
final_df.to_csv('final_df_13.csv')

In [279]:
final_df['holiday'][839] = 0

/Users/song-yeejun/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [280]:
final_df[final_df['release_date'] == '20190717']

,title,director,distributor,distributor_group,release_date,release_season,release,form,nation,screen,...,original,actor_power,director_count,director_movie_count,actor_movie_count,log_aud,before_news,after_news,holiday,dist_cat
839,라이온 킹,존 파브로,월트디즈니컴퍼니코리아 유한책임회사,B,20190717,summer,개봉영화,장편,미국,1936,...,1,0,1,1,0,15.372219,559.0,1166.0,0,B


In [ ]:
final

In [99]:
xtree = ET.fromstring(response2)
xtree

<Element 'response' at 0x000000000D031228>

In [100]:
xtree[1][0][0][1].text

'어린이날'

In [101]:
xtree[1][0][0][3].text

'20190505'

In [104]:
rows = []
for node in xtree[1][0][0]:
    n_name = node.find('dateName').text
    n_date = node.find('locdate').text
    rows.append({'name' : n_name, 'date': n_date})

AttributeError: 'NoneType' object has no attribute 'text'